## UKB Multivariate fine-mapping workflow

1. Sufficient statistics input XtX, XtY, YtY and n. We assume covariates C have been removed from X and Y. We provide a procedure to implement this.
2. GWAS summary statistics input z and R. We assume z scores have been computed after removal of covariates C.

In [ ]:
[global]
# single column file each line is the data filename
parameter: LD_folder = path('/project2/mstephens/yuxin/ukb-bloodcells/LD')
parameter: name = 'bloodcells_chr'
parameter: analysis_units = path('/project2/mstephens/yuxin/ukb-bloodcells/regions.csv')
regions = [x.strip() for x in open(analysis_units).readlines() if x.strip() and not x.strip().startswith('#')]

In [ ]:
[LDstore_rds]
input: for_each = "regions"
output: f"{LD_folder:a}/{name}{_regions[0]}.{_regions[1]}.{_regions[2]}.ld.rds"
task: trunk_workers = 1, trunk_size = 1, walltime = '12h', mem = '20G', cores = 2, tags = f'{step_name}_{_output:bn}'
R: expand = "${ }"
    ld_file = '/project2/mstephens/yuxin/ukb-bloodcells/LD_ldstore/bloodcells_chr${_regions[0]}.${_regions[1]}.${_regions[2]}.matrix'
    library(data.table)
    ld = as.matrix(fread(ld_file))
    saveRDS(ld, ${_output:r})

In [ ]:
[sufficient_stats_preprocessing]
# Phenotype file, assuming a header
parameter: LDfolder = path('')
parameter: phenoFile = path('.')
parameter: phenoCols = []
parameter: covarFile = path('.')
parameter: covarCols = []
input: for_each = "regions"
output: f"{cwd:a}/chr{_regions[0]}_{_regions[1]}_{_regions[2]}.rds"
task: trunk_workers = 1, trunk_size = 1, walltime = '12h', mem = '12G', cores = 2, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr"
    library(data.table)
    